In [1]:
import numpy as np 
import pandas as pd


from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier 

In [2]:
def read_data_from_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

In [3]:
REVIEWS_DATASET_PATH = './datasets/Womens_Clothing_E-Commerce_Reviews.csv'
reviews_dataset = read_data_from_csv(file_path=REVIEWS_DATASET_PATH)

## Resumen del dataset

El dataset contiene 23486 filas, cada una correspondiente a una reseña con unas 10 columnas correspondiente a variables explicadas a continuación.
### Descripción de cada columna

| Columna  | Descripción  |
|---|---|
| Clothing ID | Identificador numérico de la prenda especificada en la crítica |
| Age | Valor numérico correspondiente a la edad de la persona que realizo la crítica |
| Title | Titulo o encabezado de la crítica |
| Reviw Text | Contenido de la reseña realizada |
| Rating | Valor numérico en el rango del 1 al 5, siendo el 1 representante de máxima inconformidad y el 5 correspondiente al mejor valor posible |
| Recommended IND | Valor entre 0 y 1, representando respectivamente si el cliente no recomienda el producto o por el contrario si lo recomienda |
| Positive Feedback Count | Cantidad de reseñas positivas recibidas para el mismo producto |
| Division Name | Nombre de la división a la cual pertenece el producto |
| Department Name | Nombre del departamento al cual pertenece el producto |
| Class Name | Nombre de la clase al cual pertenece el producto |

### Categoria de variables

- Variables Cuantitativas : Age, Positive Feedback Count.

- Variables Cualitativas : Clothing ID, Title, Review Text, Rating, Recommended IND, Division Name, Department Name, Class Name.


In [4]:
reviews_dataset.dtypes

Unnamed: 0                  int64
Clothing ID                 int64
Age                         int64
Title                      object
Review Text                object
Rating                      int64
Recommended IND             int64
Positive Feedback Count     int64
Division Name              object
Department Name            object
Class Name                 object
dtype: object

In [5]:
reviews_dataset

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


## Limpieza de Datos

### DATOS NULOS

Observamos la cantidad de nulos que poseen cada columna y el porcentaje que representan dicha cantidad de datos en cada columna

In [6]:
reviews_dataset.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [7]:
reviews_dataset.isnull().sum() *100 / len(reviews_dataset.index)

Unnamed: 0                  0.000000
Clothing ID                 0.000000
Age                         0.000000
Title                      16.222430
Review Text                 3.597888
Rating                      0.000000
Recommended IND             0.000000
Positive Feedback Count     0.000000
Division Name               0.059610
Department Name             0.059610
Class Name                  0.059610
dtype: float64

Debido a que el trabajo a realizar con el dataset consiste en clasificar una reseña como una crítica positiva o una negativa a partir del texto en el campo “Review Text”, decidimos eliminar las columnas que no resultan de utilidad para dicho objetivo. De esta forma, obtenemos un dataset con las columnas necesarias para identificar las distintas prendas, el texto escrito en la reseña y por último la columna "Rating" para el desarrollo del ítem D, motivo por el cual conservamos dicha columna.

In [8]:
columns_eliminate= ['Unnamed: 0','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name']
reviews_dataset.drop(columns_eliminate,axis = 'columns',inplace = True)

reviews_dataset

,Clothing ID,Review Text,Rating
0,767,Absolutely wonderful - silky and sexy and comf...,4
1,1080,Love this dress! it's sooo pretty. i happene...,5
2,1077,I had such high hopes for this dress and reall...,3
3,1049,"I love, love, love this jumpsuit. it's fun, fl...",5
4,847,This shirt is very flattering to all due to th...,5
...,...,...,...
23481,1104,I was very happy to snag this dress at such a ...,5
23482,862,"It reminds me of maternity clothes. soft, stre...",3
23483,1104,"This fit well, but the top was very see throug...",3
23484,1084,I bought this dress for a wedding i have this ...,3


Por ultimo dado a que las filas con valores inválidos en la columna de "Review Text" son un total de 845 que representan una cantidad menor al 4% de la totalidad de los datos, optamos por eliminar dichas filas.

In [9]:
reviews_dataset.dropna(inplace=True)
reviews_dataset.shape

(22641, 3)

## Construccion de una nueva variable:

## Verificación:
Compruebo que la variable Rating no posea valores fuera de rango (1 al 5) antes de operar con dicha columna

In [10]:
reviews_dataset.Rating.value_counts()

5    12540
4     4908
3     2823
2     1549
1      821
Name: Rating, dtype: int64

In [11]:
reviews_dataset['Rating Category'] = reviews_dataset['Rating'].apply(lambda x: 'Negativo' if x <= 3 else 'Positivo')
reviews_dataset

,Clothing ID,Review Text,Rating,Rating Category
0,767,Absolutely wonderful - silky and sexy and comf...,4,Positivo
1,1080,Love this dress! it's sooo pretty. i happene...,5,Positivo
2,1077,I had such high hopes for this dress and reall...,3,Negativo
3,1049,"I love, love, love this jumpsuit. it's fun, fl...",5,Positivo
4,847,This shirt is very flattering to all due to th...,5,Positivo
...,...,...,...,...
23481,1104,I was very happy to snag this dress at such a ...,5,Positivo
23482,862,"It reminds me of maternity clothes. soft, stre...",3,Negativo
23483,1104,"This fit well, but the top was very see throug...",3,Negativo
23484,1084,I bought this dress for a wedding i have this ...,3,Negativo


In [12]:
reviews = reviews_dataset.loc[:, ['Review Text', 'Rating Category']]
reviews

,Review Text,Rating Category
0,Absolutely wonderful - silky and sexy and comf...,Positivo
1,Love this dress! it's sooo pretty. i happene...,Positivo
2,I had such high hopes for this dress and reall...,Negativo
3,"I love, love, love this jumpsuit. it's fun, fl...",Positivo
4,This shirt is very flattering to all due to th...,Positivo
...,...,...
23481,I was very happy to snag this dress at such a ...,Positivo
23482,"It reminds me of maternity clothes. soft, stre...",Negativo
23483,"This fit well, but the top was very see throug...",Negativo
23484,I bought this dress for a wedding i have this ...,Negativo


In [13]:
le = LabelEncoder()
reviews['Rating Category'] = le.fit_transform(reviews['Rating Category'])
reviews

,Review Text,Rating Category
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",0
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,0


In [14]:
reviews.dtypes

Review Text        object
Rating Category     int64
dtype: object

In [15]:
stop_words = set(stopwords.words('english'))
vect = CountVectorizer(stop_words=stop_words)
vect_df = vect.fit_transform(reviews['Review Text'].values.tolist())
vect_df.shape

(22641, 14005)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(vect_df, reviews['Rating Category'], test_size=0.3, random_state=10, stratify=reviews['Rating Category'])

### Naive Bayes

In [17]:
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

MultinomialNB()

In [18]:
y_nb_pred = nb_clf.predict(X_test)
report_NB = classification_report(y_test, y_nb_pred, digits=3)

### Regresión Logística

Para poder utilizar l función, Fit se deben verificar dos cosas:

i. Que no haya datos NaN

ii. Que cada elemento en la fila X_train, sea un vector.

In [19]:
lr = LogisticRegression(C=0.5, max_iter=150)
lr.fit(X_train, y_train)

LogisticRegression(C=0.5, max_iter=150)

In [20]:
y_lr_pred = lr.predict(X_test)
report_LR =classification_report(y_test, y_lr_pred, digits=3)

### Árboles de decisión

In [21]:
tree = tree.DecisionTreeClassifier(criterion='gini',min_samples_split=10,min_samples_leaf=2,ccp_alpha=0.0,max_depth=10) 

tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=10)

In [22]:
y_tree_pred = tree.predict(X_test)

report_DT = classification_report(y_test, y_tree_pred, digits=3)

### Random Forest

In [23]:
forest_model= RandomForestClassifier(criterion='gini',min_samples_split=15,min_samples_leaf=7,ccp_alpha=0.0,max_depth=12, n_estimators=15)

forest_model.fit(X_train,y_train)

RandomForestClassifier(max_depth=12, min_samples_leaf=7, min_samples_split=15,
                       n_estimators=15)

In [24]:
y_forest_pred = forest_model.predict(X_test)

report_RF = classification_report(y_test, y_forest_pred, digits=3)

## Aclaración:

En los últimos dos modelos utilizados, Árboles de decisión y Random Forest se puede implementar CrossValidation con el fin de encontrar los hiperparametros que mejoren el resultado obtenido según una determinada métrica. 

Específicamente se puede implementar un RandomSearch para el proceso de CrossValidaton y de esta forma probar una cierta cantidad de conjuntos aleatorios de hiperparametros pertenecientes a un rango que le brindamos al proceso. En su contraparte, también es posible utilizar GridSearch para probar todas las combinaciones posibles de hipeparametros entre los rangos brindados al proceso y obtener la mejor combinación.

Por ejemplo en el modelo de Random Forest es posible utilizar CrossValidation para los hipervalores: 'criterion', 'min_samples_split', 'min_samples_leaf', 'ccp_alpha' y 'max_depth'.

Sin embargo, preferimos no utilizar CrossValidation con el fin de obtener un mejor resultado a la hora de comparar los modelos, es decir, realizar la comparación con una de las posibilidades de hiperparametros y no con un conjunto que de mejores resultados.

### comparación de los modelos

In [25]:
print("Naive Bayes: \n", report_NB)
print("Regresión logística: \n",report_LR)
print("Árboles de decisión \n:",report_DT)
print("Random Forest: \n",report_RF)

Naive Bayes: 
               precision    recall  f1-score   support

           0      0.736     0.696     0.715      1558
           1      0.911     0.926     0.918      5235

    accuracy                          0.873      6793
   macro avg      0.823     0.811     0.817      6793
weighted avg      0.871     0.873     0.872      6793

Regresión logística: 
               precision    recall  f1-score   support

           0      0.758     0.635     0.691      1558
           1      0.896     0.940     0.918      5235

    accuracy                          0.870      6793
   macro avg      0.827     0.788     0.804      6793
weighted avg      0.865     0.870     0.866      6793

Árboles de decisión 
:               precision    recall  f1-score   support

           0      0.635     0.368     0.466      1558
           1      0.833     0.937     0.882      5235

    accuracy                          0.807      6793
   macro avg      0.734     0.653     0.674      6793
weighted avg 

### Implementación del mejor modelo:

Debido a que la métrica F1-score emplea los valores tanto de la precisión como del recall, optamos por darle mayor pero a dicha métrica a la hora de elegir el mejor modelo. Por este motivo optamos por utilizar Naive Bayes para catalogar a las reseñas en las 5 categorías de Rating. Dado a que no utilizaremos la columna de Rating binario “Rating Category” la eliminaremos.

In [26]:
reviews_dataset.drop(columns = 'Rating Category', inplace = True)
reviews_dataset

,Clothing ID,Review Text,Rating
0,767,Absolutely wonderful - silky and sexy and comf...,4
1,1080,Love this dress! it's sooo pretty. i happene...,5
2,1077,I had such high hopes for this dress and reall...,3
3,1049,"I love, love, love this jumpsuit. it's fun, fl...",5
4,847,This shirt is very flattering to all due to th...,5
...,...,...,...
23481,1104,I was very happy to snag this dress at such a ...,5
23482,862,"It reminds me of maternity clothes. soft, stre...",3
23483,1104,"This fit well, but the top was very see throug...",3
23484,1084,I bought this dress for a wedding i have this ...,3


In [27]:
X_train, X_test, y_train, y_test = train_test_split(vect_df, reviews_dataset['Rating'], test_size=0.3, random_state=10, stratify=reviews_dataset['Rating'])

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

y_nb_pred = nb_clf.predict(X_test)

print(classification_report(y_test, y_nb_pred, digits=3))

              precision    recall  f1-score   support

           1      0.385     0.041     0.074       246
           2      0.293     0.084     0.130       465
           3      0.392     0.426     0.408       847
           4      0.423     0.373     0.397      1473
           5      0.762     0.894     0.823      3762

    accuracy                          0.637      6793
   macro avg      0.451     0.364     0.366      6793
weighted avg      0.597     0.637     0.604      6793



### Conclusiones:




En primer lugar separamos nuevamente los datos del dataset filtrado en con una proporción del 70% y 30 % para posteriormente entrenar un modelo de Naive Bayes con el objetivo de catalogar las reseñas entre las 5 categorías de Rating.

Al analizar los resultados de las métricas en las distintas categorías, podemos observar que son bastante distintas. Por ejemplo, observamos que la metrica F1 vale 0.074 para la categoría de reseñas con 1 estrella pero sin embargo vale 0.823 para aquellas que poseen 5 estrellas. Una posible explicación para esta situación es la cantidad de datos que poseen cada categoría, ya que la categoría de 5 estrellas cuenta con 12540 reseñas contra las 821 reseñas categorizadas con 1 estrella. 
En caso de desear un mejor resultado en las categorías donde peor se desempeña el modelo, se deberán utilizar otras estrategias como por ejemplo modificar los datos del dataset (emparejando las cantidades sea con datos nuevos u operado con los datos existentes), otra opción sería el control de los conjuntos de entrenamiento del modelo para que los mismos sean balaceados.

Con Naive Bayes se obtubieron mejores resultados que con Regresión Logistíca y Árboles decisión. Esto se puede deber a que este algoritmo funciona mejor para análisis de sentimientos o que el volumen de datos no sea lo suficientemente significativo.